In [ ]:
!pip install Keras-Preprocessing
!pip install unidecode
!pip install pyvi 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.9/235.9 kB 15.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 22.7 MB/s eta 0:00:00


In [ ]:
import keras.models
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras_preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
import pickle
from pyvi import ViTokenizer
from unidecode import unidecode

In [ ]:
path = '/content/Data.csv'

data = pd.read_csv(path)
data = data[['Review','Label']]
print(data[['Review','Label']])
print()
print("data['Label'] == '1': ", (data['Label'] == -1).sum())
print("data['Label'] == '-1': ", (data['Label'] == -1).sum())

                                                 Review  Label
0     Lần đầu_tiên ăn chắc cũng là lần cuối ăn_ở đây...     -1
1     Khi mình vào thì bàn chưa dọn , dơ kinh . Cái ...     -1
2     Haiz ! Không biết đúng hương_vị của bánh thế_n...     -1
3     Mình ghé quán này vì thấy có cơm theo kiểu Việ...     -1
4     Quán xịn ở Quan_Hoa - Cầu_Giấy , biển_hiệu màu...     -1
...                                                 ...    ...
9995  Quán không_gian nhỏ nhưng khá lịch_sự , sạch_s...      1
9996  đi ăn lần 2 : ) ) ) lần này rút kinh_nghiệm ki...      1
9997  Mình ăn 2 món , mà quên tên mất rồi , toàn từ ...      1
9998  Trong ảnh là\n• 1 phần mì udon xào\n• 1 phần c...      1
9999  địa_điểm : dễ tìm , mậu thân quẹo vào đường bờ...      1

[10000 rows x 2 columns]

data['Label'] == '1':  5000
data['Label'] == '-1':  5000


In [ ]:
data['Label'] = data['Label'].astype(str)

data['Review'] = data['Review'].apply(lambda x: x.lower())
print(f"{'lower: ':<13}", data.iloc[0]['Review'])

data['Review'] = data['Review'].apply((lambda x: re.sub(r'[^a-zA-Z0-9\sÀ-ỹ]',' ',x)))
print(f"{'resub: ':<13}", data.iloc[0]['Review'])

#data['Review'] = data['Review'].apply(lambda x: ViTokenizer.tokenize(x))
#print('tokenize: ', data.iloc[0]['Review'])

data['Review'] = data['Review'].apply(lambda x: unidecode(x))
print(f"{'unidecode: ':<13}" , data.iloc[0]['Review'])

for idx,row in data.iterrows():
  row[0] = row[0].replace('rt',' ')
max_fatures =2000
X = data['Review'].values #//posts
tokenizer: Tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['Review'].values)
X = tokenizer.texts_to_sequences(data['Review'].values)
X = pad_sequences(X)
print(X)

lower:        lần đầu_tiên ăn chắc cũng là lần cuối ăn_ở đây quá , món ăn thì không có gì đặc_sắc , giá_cả thì mắc , đồ_ăn thì không được nhìu . nói_chung là mình thấy bình_thường so với một nhà_hàng tầm_cỡ như_vậy . thất_vọng ! ! ! !

resub:        lần đầu tiên ăn chắc cũng là lần cuối ăn ở đây quá   món ăn thì không có gì đặc sắc   giá cả thì mắc   đồ ăn thì không được nhìu   nói chung là mình thấy bình thường so với một nhà hàng tầm cỡ như vậy   thất vọng        

unidecode:    lan dau tien an chac cung la lan cuoi an o day qua   mon an thi khong co gi dac sac   gia ca thi mac   do an thi khong duoc nhiu   noi chung la minh thay binh thuong so voi mot nha hang tam co nhu vay   that vong        

[[   0    0    0 ...  138  139  234]
 [   0    0    0 ...  206 1082  478]
 [   0    0    0 ...   38  120  165]
 ...
 [   0    0    0 ...   60    9  116]
 [   0    0    0 ...  664   64   84]
 [   0    0    0 ...   90  330   14]]


In [ ]:
Y = pd.get_dummies(data['Label']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.3, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(7000, 1582) (7000, 2)
(3000, 1582) (3000, 2)


In [ ]:
pkl_filename = "/content/drive/Othercomputers/My Laptop/Do_An/Do_An_2/demo/model_LSTM_v310523.pkl"

with open(pkl_filename, 'rb') as file:
  load_model_LSTM = pickle.load(file)

In [ ]:
validation_size = 880
X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
print(X_validate.shape,Y_validate.shape)
print(X_test.shape,Y_test.shape)

(880, 1582) (880, 2)
(2120, 1582) (2120, 2)


In [ ]:
print(X_test.shape,Y_test.shape)
score, acc = load_model_LSTM.evaluate(X_test, Y_test, verbose=1, batch_size=32)
print('Điểm đánh giá: %.2f', (score * 100), '%')
print('Độ chính xác: %.2f' % (acc * 100), '%')

In [ ]:
from numpy.lib.npyio import load
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0

for x in range(len(X_validate)):
    result = load_model_LSTM.predict(X_validate[x].reshape(1, X_test.shape[1]), batch_size=1, verbose=0)[0]
    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1

    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1

print("Độ chính xác với bình luận tích cực:", pos_correct / pos_cnt * 100, "%")
print("Độ chính xác với bình luận tiêu cực:", neg_correct / neg_cnt * 100, "%")

Độ chính xác với bình luận tích cực: 85.26570048309179 %
Độ chính xác với bình luận tiêu cực: 81.54506437768241 %


In [ ]:
# Test bình luận là positive

twt = ['Lần đầu tiên vào về ngay lập tức bị nghiện trà đào và bông lan trứng muối chắc ngày nào cũng phải ra đây thôi']
twt = tokenizer.texts_to_sequences(twt)
twt = pad_sequences(twt, maxlen=28, dtype='int32', value=0)
print(twt)

sentiment = load_model_LSTM.predict(twt, batch_size=1, verbose=2)[0]
if np.argmax(sentiment) == 0:
    print("[{}] Tiêu cực".format(np.argmax(sentiment)))
elif np.argmax(sentiment) == 1:
    print("[{}] Tích cực".format(np.argmax(sentiment)))
print(sentiment)

In [ ]:
reviews = [
    "Chất lượng món ăn và dịch vụ tại quán thật sự xuất sắc. Món ăn thơm ngon, tươi ngon và hấp dẫn. Nhân viên phục vụ rất nhiệt tình và chu đáo. Tôi rất hài lòng với trải nghiệm ở đây.",
    "Quán này có chất lượng món ăn và phục vụ tuyệt vời. Mọi thứ đều rất chuyên nghiệp và ngon miệng. Nhân viên rất thân thiện và luôn sẵn sàng phục vụ khách hàng. Tôi rất thích đến đây để thưởng thức các món ăn ngon.",
    "Tôi rất ấn tượng với chất lượng món ăn và phục vụ tại quán. Món ăn được chế biến tươi ngon, hương vị đậm đà. Nhân viên phục vụ rất nhanh nhẹn và thân thiện. Tôi sẽ quay lại quán này trong tương lai.",
    "Món ăn tại quán thực sự ngon và đa dạng. Chất lượng của các món ăn rất tốt, với những thành phần tươi ngon và chế biến chuyên nghiệp. Dịch vụ tại quán cũng rất tốt, nhân viên phục vụ nhanh nhẹn và thân thiện.",
    "Quán này thực sự đáng để khám phá với chất lượng món ăn và phục vụ tuyệt vời. Món ăn rất ngon, tươi ngon và được chế biến tỉ mỉ. Nhân viên phục vụ luôn niềm nở và tận tâm. Tôi hài lòng với trải nghiệm tại đây.",
    "Tôi rất hài lòng với chất lượng món ăn và phục vụ tại quán. Món ăn ngon, tươi ngon và thực sự đáng thử. Nhân viên phục vụ rất chu đáo và tận tâm. Tôi sẽ khuyên bạn bè và người thân đến quán này.",
    "Tuyệt vời! Chất lượng món ăn và dịch vụ ở quán này thật tệ. Món ăn không ngon, không tươi ngon và không đáng giá. Nhân viên phục vụ cũng không lịch sự và không chu đáo. Tôi không hài lòng với trải nghiệm ở đây.",
    "Tôi thực sự thất vọng với chất lượng món ăn và phục vụ tại quán. Món ăn thiếu hương vị và không chất lượng. Dịch vụ cũng chậm và không chuyên nghiệp. Tôi không khuyến cáo đến quán này.",
    "Quán này không đáng để ghé thăm với chất lượng món ăn và dịch vụ kém. Món ăn không ngon, không tươi ngon và không đáng giá. Nhân viên phục vụ cũng không lịch sự và không thân thiện. Tôi không sẽ quay lại đây.",
    "Món ăn tại quán thực sự tệ. Chất lượng không đáng giá và không ngon. Dịch vụ cũng kém, nhân viên phục vụ không chu đáo và thiếu chuyên nghiệp. Tôi không hài lòng với trải nghiệm tại đây."
]


for txt in reviews:
  print(txt)
  txt = tokenizer.texts_to_sequences(txt)
  txt = pad_sequences(txt, maxlen=28, dtype='int32', value=0)
  sentiment = load_model_LSTM.predict(txt, batch_size=1, verbose=0)[0]
  if np.argmax(sentiment) == 0:
      print("[{}] Tiêu cực".format(np.argmax(sentiment)))
  elif np.argmax(sentiment) == 1:
      print("[{}] Tích cực".format(np.argmax(sentiment)))
  print('-'*80)

Chất lượng món ăn và dịch vụ tại quán thật sự xuất sắc. Món ăn thơm ngon, tươi ngon và hấp dẫn. Nhân viên phục vụ rất nhiệt tình và chu đáo. Tôi rất hài lòng với trải nghiệm ở đây.
[1] Tích cực
--------------------------------------------------------------------------------
Quán này có chất lượng món ăn và phục vụ tuyệt vời. Mọi thứ đều rất chuyên nghiệp và ngon miệng. Nhân viên rất thân thiện và luôn sẵn sàng phục vụ khách hàng. Tôi rất thích đến đây để thưởng thức các món ăn ngon.
[0] Tiêu cực
--------------------------------------------------------------------------------
Tôi rất ấn tượng với chất lượng món ăn và phục vụ tại quán. Món ăn được chế biến tươi ngon, hương vị đậm đà. Nhân viên phục vụ rất nhanh nhẹn và thân thiện. Tôi sẽ quay lại quán này trong tương lai.
[1] Tích cực
--------------------------------------------------------------------------------
Món ăn tại quán thực sự ngon và đa dạng. Chất lượng của các món ăn rất tốt, với những thành phần tươi ngon và chế biến chuyên